In [5]:
import boto3
import redis
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
import json

ModuleNotFoundError: No module named 'redis'

In [7]:
# DynamoDB 및 Redis 클라이언트 초기화
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('merged-data-table')

# r = redis.StrictRedis(host='221.151.189.183', port=6379, db=0)

In [24]:
import os

os.getcwd()

FileNotFoundError: [Errno 2] No such file or directory

In [22]:
def access_keys(path):
    with open(f'{path}/', 'r') as f:
        key = json.load(f)
    return key

In [8]:
# DynamoDB에서 id 값을 가져오는 함수
def get_ids_from_dynamodb(key):
    AWS_ACCESS_KEY = key['aws_access_key_id']
    AWS_SECRET_KEY = key['aws_secret_key']
    REGION = key['region']
    
    response = table.scan()  # Scan을 통해 DynamoDB 테이블에서 모든 항목을 가져옴
    items = response.get('Items', [])
    return [item['id'] for item in items if 'id' in item]  # 'id' 필드만 추출

# Redis에 id 값을 저장하는 함수
def save_to_redis(ids):
    for id_value in ids:
        if not r.hexists('compare_id', id_value):  # Redis hash에 해당 id가 없으면
            r.hset('compare_id', id_value, 1)  # 해당 id를 해시에 저장
            print(f"새로운 id {id_value}가 Redis에 추가되었습니다.")
        else:
            print(f"id {id_value}는 이미 Redis에 존재합니다.")

In [9]:
def check_dynamodb_connection(key):
    AWS_ACCESS_KEY = key['aws_access_key_id']
    AWS_SECRET_KEY = key['aws_secret_key']
    REGION = key['region']
    
    try:
        dynamodb = boto3.client('dynamodb',
            region_name=REGION,
            aws_access_key_id=AWS_ACCESS_KEY,
            aws_secret_access_key=AWS_SECRET_KEY)
        
        
        # DynamoDB에서 테이블 목록을 가져옴
        response = dynamodb.list_tables()
        table_names = response.get('TableNames', [])
        
        if table_names:
            print(f"DynamoDB에 성공적으로 연결되었습니다. 테이블 목록: {table_names}")
        else:
            print("DynamoDB에 연결되었으나 테이블이 없습니다.")
    except NoCredentialsError:
        print("AWS 자격 증명이 제공되지 않았습니다. 자격 증명을 확인하세요.")
    except PartialCredentialsError:
        print("AWS 자격 증명이 불완전합니다. 자격 증명을 확인하세요.")
    except Exception as e:
        print(f"DynamoDB 연결 중 오류가 발생했습니다: {str(e)}")

In [10]:
def main():
    path = '/home/team3/repository/keys/'
    key = access_keys(path)
    #ids = get_ids_from_dynamodb()
    check_dynamodb_connection(key)
    #save_to_redis(ids)

In [11]:
if __name__ == "__main__":
    main()

NameError: name 'access_keys' is not defined

In [27]:
import re, datetime, pytz, subprocess, os, json
from botocore.exceptions import ClientError
from time import gmtime, strftime
import boto3



# DynamoDB 설정
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')

# 테이블 이름 설정
table_name = 'merged-data-table'
table = dynamodb.Table(table_name)

def get_item_from_dynamodb(key):
    try:
        response = table.get_item(Key=key)
        if 'Item' in response:
            return response['Item']
        else:
            print("Item not found")
            return None
    except Exception as e:
        print(f"Error getting item: {e}")
        return None


def main():
    # 예시 키 값
    key = {'id': 'example_id'}
    
    # DynamoDB에서 아이템 가져오기
    item = get_item_from_dynamodb(key)
    if item:
        print("Item found:", item)
    else:
        print("Item not found")






# # s3 client를 통해 해당 path의 file 목록 가져오기
# def get_bucket_metadata(s3_client, pull_bucket_name, target_folder_prefix):
#     # 특정 폴더 내 파일 목록 가져오기
#     response = s3_client.list_objects_v2(Bucket=pull_bucket_name, Prefix=target_folder_prefix, Delimiter='/')

#     if 'Contents' in response:
#         return [obj for obj in response['Contents']]
#     else:
#         #print("No objects found in the folder.")
#         return None

# # 로컬 시간대(UTC+9)로 현재 날짜 설정
# def get_curr_kst_time():
#     return datetime.datetime.now(pytz.timezone('Asia/Seoul')).date()

# # kst timezone 설정
# def set_kst_timezone():
#     return pytz.timezone('Asia/Seoul')

# # 파싱을 위해 unuseal line terminators 제거
# def remove_unusual_line_terminators(text):
#     return re.sub(r'[\r\u2028\u2029]+', ' ', text)

# # 정규 표현식을 사용하여 한글, 영어 알파벳, 숫자, 공백을 제외한 모든 문자를 공백으로 치환
# def replace_special_to_space(text, pattern=r'[^a-zA-Z0-9가-힣\s]'):
#     return re.sub(pattern, ' ', text)

# def remove_multiful_space(text):
#     return (' '.join(text.split())).strip()

# def change_slash_format(text):
#     return text.replace(" /", ",").replace("/", ",")

# def change_str_to_timestamp(text):
#     if text:
#         return str(int(datetime.datetime.strptime(text, "%Y-%m-%d").timestamp()))
#     else:
#         return None

# # unique id를 보관하는 S3 버킷의 obj_ids.json 업데이트
# def update_ids_to_s3(s3_client, buket_name, prefix, upload_id_list):
#     odd_id_list = get_id_from_s3(s3_client, buket_name, "obj_ids.json")
#     new_id_list = list(set(odd_id_list + upload_id_list))
#     update_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
#     data = {
#         "version": "2024-09-04",
#         "statement": "id_list_of_precessed_data_objects",
#         "ids": new_id_list,
#         "last_update": update_date
#     }
#     json_string = json.dumps(data)
#     response = s3_client.put_object(
#         Bucket=buket_name,          # 버킷 이름
#         Key=prefix,             # 업로드할 파일의 키(경로 및 이름)
#         Body=json_string,               # 업로드할 파일의 내용
#         ContentType='application/json'  # 파일의 MIME 타입
#     )
#     return response

# # unique id를 보관하는 S3 버킷의 obj_ids.json에서 id 목록 호출
# def get_id_from_s3(s3_client, buket_name, prefix):
#     metadata_list = get_bucket_metadata(s3_client, buket_name, prefix)
#     if metadata_list:
#         try:
#             _obj = metadata_list[0]
#             response = s3_client.get_object(Bucket=buket_name, Key=_obj['Key'])
#             json_context = response['Body'].read().decode('utf-8')
#             join_dict = json.loads(json_context)
#             return join_dict.get('ids')
#         except json.JSONDecodeError as e:
#             #logging.error(f"JSONDecodeError encountered: {e}")
#             return False
#         except ClientError as e:
#             #logging.error(f"ClientError encountered: {e}")
#             return False
#         except Exception as e:
#             #logging.error(f"Unknow Error. encountered: {e}")
#             return False

# # unique id 목록과 현재 df의 id column을 비교하여 unique id 목록에 없는 id column 값만 list로 반환
# def remove_duplicate_id(s3_client, buket_name, _df):
#     id_list = get_id_from_s3(s3_client, buket_name, "obj_ids.json")
#     df_id_list = _df['id'].unique().tolist()
#     if id_list:
#         unput_id_list = [id for id in df_id_list if id not in id_list]
#         return unput_id_list
#     else:
#         return df_id_list

# # redis에서 id가 존재하는지 체크 후 없는 것들에 대해서 record 반환
# def check_id_in_redis(logger, redis_sassion, records):
#     inited_id_records = []
#     for record in records:
#         _id = record.get('id')
#         _get_date = record.get('get_date')
#         if not redis_sassion.hexists('compare_id', _id):  # Redis hash에 해당 id가 없으면
#             inited_id_records.append({"id": _id, "get_date": _get_date})
#             logger.info(f"id {_id} is not exist in redis. Added upload list.")
#         else:
#             logger.info(f"id {_id} already exist in redis. This id passed.")
    
#     return inited_id_records

# # record에서 없는 id만 redis로 push
# def upload_id_into_redis(logger, redis_sassion, records):
#     for record in records:
#         _id = record.get('id')
#         _get_date = record.get('get_date')
#         if not redis_sassion.hexists('compare_id', _id):  # Redis hash에 해당 id가 없으면
#             redis_sassion.hset('compare_id', _id, _get_date)  # 해당 id를 해시에 저장
#             logger.info(f"id {_id} init into redis.")
#         else:
#             logger.info(f"id {_id} already exist in redis. set id action dismissed")




In [28]:

if __name__ == "__main__":
    main()

Error getting item: Unable to locate credentials
Item not found
